In [1]:
import warnings

# Filter specific warning
warnings.filterwarnings("ignore")

In [21]:
import numpy as np
import pandas as pd
import random

df_org = pd.read_csv('../Dataset/kl.csv')
df_org = df_org.head(10000)

# Selecting the relevant variables
col = ['Age', 'Overall', 'Potential', 'Wage', 'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Height', 'Weight', 'Club', 'Value']
df = df_org[col]

df.columns = ['Age', 'Overall', 'Potential', 'Wage', 'Special', 'PreferredFoot', 'InternationalReputation', 'WeakFoot',
              'SkillMoves', 'Height', 'Weight', 'Club', 'Value']


In [22]:
# Changing the format of several string variabls
val = []
for i in df['Value']:
    if i[-1] == 'M':
        val.append(float(i[1:-1]) * 1000000)
    elif i[-1] == 'K':
        val.append(float(i[1:-1]) * 1000)
    else:
        val.append(0)
df.loc[:, 'Value'] = val

val = []
for i in df['Wage']:
    if i[-1] == 'M':
        val.append(float(i[1:-1]) * 1000000)
    elif i[-1] == 'K':
        val.append(float(i[1:-1]) * 1000)
    else:
        val.append(0)
df.loc[:, 'Wage'] = val

height = df.Height.str.split('\'', 1)
height0 = []
height1 = []
for i in height:
    if type(i) is list:
        height0.append(i[0])
        height1.append(i[1])
    else:
        height0.append(0)
        height1.append(0)

height_cm = []
for i in range(len(height)):
    height_cm.append(round((float(height0[i]) * 30.48 + float(height1[i]) * 2.54) / 100, 3))
df.loc[:, 'Height'] = height_cm

df['Weight'] = pd.to_numeric(df['Weight'].str.rstrip('lbs'))

df = df.iloc[np.where((df['Value'] > 0) & (df['Height'] > 0))]

df.rename({'Club': 'cluster'}, axis=1, inplace=True)
df.rename({'Value': 'y'}, axis=1, inplace=True)

In [23]:
# енкодинг
df['PreferredFoot'] = pd.Categorical(df['PreferredFoot']).codes
df['cluster'] = pd.Categorical(df['cluster']).codes
codes = pd.Categorical(df['cluster']).unique()

In [24]:
codes

[206, 316, 421, 363, 362, ..., 154, 353, 559, 569, 121]
Length: 630
Categories (630, int64): [206, 316, 421, 363, ..., 353, 559, 569, 121]

In [25]:
# нормалізувати (наблизити до 0)
min(df['y'])

40000.0

In [38]:
min(df['Wage'])

1.0

In [26]:
df['y'] = df['y'] / 10000
df['Wage'] = df['Wage'] / 1000

In [59]:
df.head(7)

,Age,Overall,Potential,Wage,Special,PreferredFoot,InternationalReputation,WeakFoot,SkillMoves,Height,Weight,cluster,y
0,31.0,94.0,94,565.0,2202,0,5.0,4.0,4.0,1.702,159,206,11050.0
1,33.0,94.0,94,405.0,2228,1,5.0,4.0,5.0,1.880,183,316,7700.0
2,26.0,92.0,93,290.0,2143,1,5.0,5.0,5.0,1.753,150,421,11850.0
3,27.0,91.0,93,260.0,1471,1,4.0,3.0,1.0,1.930,168,363,7200.0
4,27.0,91.0,92,355.0,2281,1,4.0,5.0,4.0,1.803,154,362,10200.0
5,27.0,91.0,91,340.0,2142,1,4.0,4.0,4.0,1.727,163,132,9300.0
6,32.0,91.0,91,420.0,2280,1,4.0,4.0,4.0,1.727,146,455,6700.0


## Train Test split

In [28]:
## Дані будуть розділені на тренувальну і тестувальну вибірки, так щоб не було однакових значень 'cluster' між цими вибірками.
version = 'CV'
depth = 5
NoCov = 2

In [60]:
labels = np.array(df['y'])
features= df.drop('y', axis = 1)

feature_list = list(features.columns)
features = np.array(features)

In [76]:
missing_values = df.isna().sum()
print(missing_values)
df = df.dropna()

Age                        1
Overall                    1
Potential                  0
Wage                       0
Special                    0
PreferredFoot              0
InternationalReputation    0
WeakFoot                   0
SkillMoves                 0
Height                     0
Weight                     0
cluster                    0
y                          0
dtype: int64


In [77]:
sample_clusters = random.sample(set(df.cluster.unique()), 20)
df_tr = df.loc[df['cluster'].isin(sample_clusters)]
train_labels = np.array(df_tr['y'])

df_test = df.loc[~df['cluster'].isin(sample_clusters)]
test_labels = np.array(df_test['y'])

print('Training Features Shape:', df_tr.shape)
print('Training Labels Shape:', df_tr['y'].shape)
print('Testing Features Shape:', df_test.shape)
print('Testing Labels Shape:', df_test['y'].shape)

Training Features Shape: (255, 13)
Training Labels Shape: (255,)
Testing Features Shape: (9582, 13)
Testing Labels Shape: (9582,)


## The analysis

# RETCO

In [88]:
from RealData import ClusteredRETCO as RETCOfile

RETCO = RETCOfile.CreateTree(version, df_tr, depth=depth, min_leaf_sample=3, StoppingRule='Yes', random_subspace=None)

best overall metric: 230714.49005679475
best overall metric: 229540.93205734057
best overall metric: 220054.1728340568
best overall metric: 202180.17825487175
best overall metric: 184596.73492102989
best overall metric: 162967.3749290129
best overall metric: 147834.2334772708
best overall metric: 134570.67698313258
best overall metric: 113712.6135013153
best overall metric: 94497.15832745167
best overall metric: 93891.83279461485
best overall metric: 85987.79670870859
best overall metric: 81862.62735730717
best overall metric: 76159.04713686835
best overall metric: 70158.3915330004
best overall metric: 63729.709260206764
best overall metric: 59852.80111240931
best overall metric: 59453.93173503757
best overall metric: 58402.52847123515
best overall metric: 58116.13852030226
best overall metric: 57897.25638279577
best overall metric: 57821.31323354506
best overall metric: 57775.98998187898
best overall metric: 57678.722330181066
best overall metric: 57578.13307472781
best overall metric

In [91]:
RETCO

{"(pred['Overall']<=77.0)*(pred['Overall']<=72.0)*(pred['Overall']<=70.0)*(pred['Potential']<=69.0)*(pred['Age']<=30.0)": 75.19841269841271,
 "(pred['Overall']>77.0)*(pred['Potential']<=80.0)*(pred['Age']<=29.0)": 1210.0,
 "(pred['Overall']<=77.0)*(pred['Overall']>72.0)*(pred['Potential']<=74.0)*(pred['Age']<=33.0)*(pred['Wage']<=20.0)": 318.57142857142856,
 "(pred['Overall']>77.0)*(pred['Potential']>80.0)*(pred['Overall']<=80.0)*(pred['WeakFoot']<=3.0)": 1450.0,
 "(pred['Overall']<=77.0)*(pred['Overall']>72.0)*(pred['Potential']>74.0)*(pred['Age']<=31.0)*(pred['Overall']<=73.0)": 523.3333333333333,
 "(pred['Overall']<=77.0)*(pred['Overall']<=72.0)*(pred['Overall']>70.0)*(pred['Age']<=27.0)*(pred['Potential']<=77.0)": 310.0,
 "(pred['Overall']>77.0)*(pred['Potential']>80.0)*(pred['Overall']>80.0)*(pred['Special']<=2054.0)": 2533.333333333333,
 "(pred['Overall']>77.0)*(pred['Potential']<=80.0)*(pred['Age']>29.0)": 602.0000000000001,
 "(pred['Overall']<=77.0)*(pred['Overall']>72.0)*(pred

In [102]:
covariates_pred_test = df_test.iloc[:, :-NoCov].values
col_names = df_test.iloc[:, :-NoCov].columns

In [103]:
prediction_test_RETCO = RETCOfile.predictionFun(col_names, covariates_pred_test, RETCO)

In [104]:
col_names

Index(['Age', 'Overall', 'Potential', 'Wage', 'Special', 'PreferredFoot',
       'InternationalReputation', 'WeakFoot', 'SkillMoves', 'Height',
       'Weight'],
      dtype='object')

In [108]:
y_test = df_test['y'].values

In [110]:
prediction_test_RETCO = RETCOfile.predictionFun(col_names, covariates_pred_test, RETCO)

In [134]:
errors2 = abs(prediction_test_RETCO - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute RETCO Error:', round(np.mean(errors2), 2), 'dollars.')

Mean Absolute RETCO Error: 114.92 dollars.


In [138]:
mape = 100 * (errors2 / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 69.44 %.


In [129]:
# Convert list to DataFrame
df1y_test = pd.DataFrame(prediction_test_RETCO, columns=['values'])

# Save DataFrame to a CSV file
df1y_test.to_csv('prediction_test_RETCO.csv', index=False)

In [116]:
from RealData import CHRT as RTfile # regretion tree file

tree = RTfile.decision_tree_algorithm(version, df_tr, max_depth=depth, min_leaf_sample=3, counter=0,
                                      random_subspace=None)

best_overall_metric 221454.80406096362
best_overall_metric 218275.35720490178
best_overall_metric 216962.51256081107
best_overall_metric 207097.73019874888
best_overall_metric 190038.20508334378
best_overall_metric 173991.88709287462
best_overall_metric 155204.50210429734
best_overall_metric 141602.50721314384
best_overall_metric 129940.58259676183
best_overall_metric 112477.12813493956
best_overall_metric 94302.56190140641
best_overall_metric 94225.43332245386
best_overall_metric 83254.45518318795
best_overall_metric 45452.20359669738
best_overall_metric 44886.344348936895
best_overall_metric 44626.747793297276
best_overall_metric 44611.17500106543
best_overall_metric 44583.17819307424
best_overall_metric 44120.94338769204
best_overall_metric 40382.168088115155
best_overall_metric 35331.58220156338
best_overall_metric 30159.867154745276
best_overall_metric 24012.753206510435
best_overall_metric 20528.782233390993
best_overall_metric 20055.675943747912
best_overall_metric 18743.0717608

In [118]:
prediction_test = RTfile.decision_tree_predictions(df_test, tree)

In [124]:
# Convert list to DataFrame
df1y_test = pd.DataFrame(prediction_test, columns=['values'])

# Save DataFrame to a CSV file
df1y_test.to_csv('prediction_test_RTfile.csv', index=False)

In [119]:
new_y_test = y_test[1:]

In [127]:
# Convert list to DataFrame
df1y_test = pd.DataFrame(y_test, columns=['y_test'])

# Save DataFrame to a CSV file
df1y_test.to_csv('y_test.csv', index=False)

In [133]:
errors3 = abs(prediction_test - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute RTerror:', round(np.mean(errors3), 2), 'dollars.')

Mean Absolute RTerror: 112.49 dollars.


In [139]:
mape = 100 * (errors3 / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 69.9 %.
